# dvc-demo

This notebook is a demonstration of using DVC on Spell. To learn more refer to the accompanying blog post: `$LINK`.

## getting the data

For the purposes of this demonstration, we will use a sample from the dataset [A Year of Pumpkin Prices](https://www.kaggle.com/usda/a-year-of-pumpkin-prices) on Kaggle (specifically, `new-york_9-24-2016_9-30-2017.csv`).

To begin, I downloaded the data to the file `new_york_pumpkin_prices.csv` on my local machine (if you are running this code in a Spell workspace, you can `spell upload` this file, then mount it into your workspace):

In [5]:
import pandas as pd
pd.read_csv("new_york_city_pumpkin_prices.csv").head()

,Commodity Name,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,...,Color,Environment,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode
0,PUMPKINS,NEW YORK,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,150,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,PUMPKINS,NEW YORK,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,150,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,PUMPKINS,NEW YORK,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,130,150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,PUMPKINS,NEW YORK,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,130,150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,PUMPKINS,NEW YORK,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,120,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


This is a simple dataset of pumpkin prices (`Low Price`, `High Price`) by `Variety` and `Item Size` sold in New York City in the days prior to Halloween 2016.

## versioning it locally using dvc

To initialize DVC, run `dvc init` (we use `--subdir` here because this demo is specific to the `dvc` directory in `spellml/examples` only). To add data to DVC, run `dvc add`.

In [7]:
!dvc init --subdir


You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [10]:
%ls

devc-demo.ipynb                   new_york_city_pumpkin_prices.csv


In [11]:
!dvc add new_york_city_pumpkin_prices.csv

100% Add|██████████████████████████████████████████████|1/1 [00:00,  1.86file/s]

To track the changes with git, run:

	git add new_york_city_pumpkin_prices.csv.dvc .gitignore


Running `dvc add` does three things. First, it creates a `[...].dvc` file containing the MD5 content hash of the file.

In [12]:
%ls

devc-demo.ipynb                       new_york_city_pumpkin_prices.csv.dvc
new_york_city_pumpkin_prices.csv


In [13]:
!cat new_york_city_pumpkin_prices.csv.dvc

outs:
- md5: 10ac52bb2b805fe1a9de704d2f5a5be1
  size: 11875
  path: new_york_city_pumpkin_prices.csv


Second, it adds the actual CSV file itself to your `.gitignore`. DVC will handle version controlling the data now instead of `git`.

In [14]:
!cat .gitignore

/new_york_city_pumpkin_prices.csv


Third, it stores a copy of the dataset in a [content addressable storage](https://en.wikipedia.org/wiki/Content-addressable_storage) filesystem inside the special `.dvc` directory.

In [16]:
%ls .dvc/cache/

10/


In [18]:
%ls .dvc/cache/10/

ac52bb2b805fe1a9de704d2f5a5be1


In [19]:
!head .dvc/cache/10/ac52bb2b805fe1a9de704d2f5a5be1

Commodity Name,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,Mostly High,Origin,Origin District,Item Size,Color,Environment,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,150,170,150,170,MICHIGAN,,xlge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,150,170,150,170,MICHIGAN,,lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,130,150,130,150,NEW JERSEY,,xlge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,130,150,130,150,NEW JERSEY,,lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,120,140,120,140,NEW YORK,,med-lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,150,170,150,170,PENNSYLVANIA,,xlge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,120,170,120,170,PENNSYLVANIA,,lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,1

At this point, `dvc checkout` can be used to rehydrate `new_york_city_pumpkin_prices.csv`. This is helpful should be lose the file, or perhaps revert to an old commit that used an older version of this dataset.

In [23]:
!rm new_york_city_pumpkin_prices.csv

In [24]:
!dvc checkout

A	new_york_city_pumpkin_prices.csv                                    


## versioning it remotely using dvc

So far everything we have done has used local storage. However, DVC's most interesting feature&mdash;it's "killer app", if you will&mdash;is the ability to use remote storage.

To do this, you configure and use a **remote**. A remote in a DVC is a cloud-based entity&mdash;an S3 or GCS bucket, typically&mdash;that serves as the central storage space for your project's data.

Setting a remote is easy to do, using `dvc remote add`:

In [38]:
!dvc remote add spell-datasets-share s3://spell-datasets-share/dvc/

In [41]:
# This line is only needed if you want DVC to use the non-default AWS profile.
!dvc remote modify spell-datasets-share profile spell2

This adds a remote with this given name and URL to the project's DVC configuration file, `.dvc/config`:

In [42]:
!cat .dvc/config

['remote "spell-datasets-share"']
    url = s3://spell-datasets-share/dvc/
    profile = spell2


We can now use `dvc push` to send that data to S3:

In [43]:
!dvc push --remote spell-datasets-share

  0% Uploading|                                      |0/1 [00:00<?,     ?file/s]
!
  0%|          |new_york_city_pumpkin_prices.cs0.00/11.6k [00:00<?,        ?B/s]
100%|██████████|new_york_city_pumpkin_pric11.6k/11.6k [00:00<00:00,    93.4kB/s]
1 file pushed                                                                   


In [44]:
!aws s3 ls s3://spell-datasets-share/dvc/ --profile spell2

                           PRE 10/


In [45]:
!aws s3 ls s3://spell-datasets-share/dvc/10/ --profile spell2

2021-01-26 11:15:46      11875 ac52bb2b805fe1a9de704d2f5a5be1


We can now easily rehydrate our dataset at any time from remote storage using `dvc pull`:

In [46]:
!rm new_york_city_pumpkin_prices.csv

In [47]:
!dvc pull --remote spell-datasets-share

A	new_york_city_pumpkin_prices.csv                                    
1 file added


In [48]:
!head new_york_city_pumpkin_prices.csv

Commodity Name,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,Mostly High,Origin,Origin District,Item Size,Color,Environment,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,150,170,150,170,MICHIGAN,,xlge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,150,170,150,170,MICHIGAN,,lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,130,150,130,150,NEW JERSEY,,xlge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,130,150,130,150,NEW JERSEY,,lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,120,140,120,140,NEW YORK,,med-lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,150,170,150,170,PENNSYLVANIA,,xlge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,120,170,120,170,PENNSYLVANIA,,lge,,,,,,,,,N,
PUMPKINS,NEW YORK,,36 inch bins,HOWDEN TYPE,,,09/24/2016,1

Now *anyone on the team* can rehydrate the complete state of the repository, using Git for the code and DVC for the data, in just a couple of commands. ✨

```bash
$ git pull
$ dvc pull
```

**We are, in essence, using DVC as a lightweight feature store.** Nifty!

## using dvc with spell

Spell has the ability to mount S3 and/or GCS buckets directly to the machine using our `--mount` syntax.